In [1]:
import os
import polars as pl
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras import layers, models, optimizers, regularizers, callbacks
import kaggle_evaluation.jane_street_inference_server

In [2]:
# model1 = tf.keras.models.load_model("/kaggle/input/tfmodels/tensorflow2/default/1/tf_nn_model3.keras")
# model2 = tf.keras.models.load_model("/kaggle/input/tfmodels/tensorflow2/default/1/tf_nn_model4_no_concat.keras")

29/70
loss: 1.6219 - r2_score: 0.0212 - val_loss: 1.1182 - val_r2_score: 0.0049 - learning_rate: 1.0000e-05


R² Score on validation data: 0.004866182804107666

In [3]:

model_path = [
#     "/kaggle/input/tfmodels/tensorflow2/default/1/tf_nn_model3.keras",
    "/kaggle/input/tf_nn_lagfeatures/tensorflow2/default/1/tf_nn_model12_lagfeatures.keras"
]

models = []
for pt in model_path:
    _model = tf.keras.models.load_model(pt)
    models.append(_model)

In [4]:
# file_path = "/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet/date_id=0/part-0.parquet"
# lazy_df = pl.scan_parquet(file_path)
# test = lazy_df.collect()  # Collect into a Polars DataFrame
# file_path = "/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet/date_id=0/part-0.parquet"
# lazy_df = pl.scan_parquet(file_path)
# lags = lazy_df.collect()  # Collect into a Polars DataFrame

In [5]:

# )

In [6]:

lags_ : pl.DataFrame | None = None

def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    global lags_, models, lag_feature_original, lag_feature_rename # 使用全局变量 lags_
    

    if lags is not None:
        lags_ = lags
        
    lag_feature_original = ["date_id", "time_id", "symbol_id"]  + [f"feature_{i:02d}" for i in range(79)]
    lag_feature_rename = { f"feature_{idx:02d}" : f"feature_{idx:02d}_lag_1" for idx in range(79)}
    feature_names = [f"feature_{i:02d}" for i in range(79)] + [f"feature_{i:02d}_lag_1" for i in range(79)] + [f"responder_{idx}_lag_1" for idx in range(9)]


    lag_feature = test.select(pl.col(lag_feature_original))
    lag_feature = lag_feature.rename(lag_feature_rename)
    lag_feature = lag_feature.with_columns(
        time_id = pl.col('time_id') + 1,
    )
    lag_feature = lag_feature.group_by(["date_id", "time_id" ,"symbol_id"], maintain_order=True).last()  # pick up last record of previous date
    test = test.join(lag_feature, on=["date_id", "time_id" ,"symbol_id"],  how="left")
    predictions = test.select(
        'row_id',  # 选择 `row_id` 列
        pl.lit(0.0).alias('responder_6'), 
    )


    if not lags is None:
        lags = lags.group_by(["date_id", "symbol_id"], maintain_order=True).last()  
        test = test.join(lags, on=["date_id", "symbol_id"], how="left") 
    else:

        test = test.with_columns(
            (pl.lit(0.0).alias(f'responder_{idx}_lag_1') for idx in range(9))  # 创建 9 个滞后特征，默认值为 0
        )

    preds = np.zeros((test.shape[0],))


    symbol_ids = test.select('symbol_id').to_numpy()[:, 0]
    test_input = test[feature_names].to_pandas()
    test_input = test_input.bfill().ffill().fillna(0)  

    for md in models:
        preds += (md.predict(test_input[feature_names]).ravel() / len(models))
        
    predictions = test.select('row_id').\
    with_columns(
        pl.Series(
            name='responder_6', 
            values=np.clip(preds, a_min=-5, a_max=5),  
            dtype=pl.Float64, 
        )
    )

    assert isinstance(predictions, pl.DataFrame | pd.DataFrame)
    assert list(predictions.columns) == ['row_id', 'responder_6']
    assert len(predictions) == len(test)
    
    return predictions  # 返回最终的预测结果

In [7]:
# Setup the inference server
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

# Running the inference server
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway((
        '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
        '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
    ))

I0000 00:00:1731986534.803688      68 service.cc:145] XLA service 0x7a6fb0008760 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731986534.803729      68 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1731986534.803733      68 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


1/2 ━━━━━━━━━━━━━━━━━━━━ 1s 2s/step

I0000 00:00:1731986536.128009      68 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 394ms/step
